In [1]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scripts.pdf_parser_class import PDFparser

## Парсинг pdf
1. Обзор некоторых доступных модулей для парсинга pdf - см. [ноутбук](scripts/notebooks/compare_modules.ipynb)
2. PDFparser - кастомный модуль, который парсит pdf в текст с помощью *pdftotext* или *camelot*, а затем распознает ряды и заголовок твблицы на основе [некоторых эвристик](../../README.md#evr)

In [2]:
NEW_PDF = "../../task_description/examples/sysco PO#_077-2706434.pdf"

In [3]:
pdftotext_parser = PDFparser()
ptt_df = pdftotext_parser.get_rows(NEW_PDF)

camelot_parser = PDFparser(parse_method="camelot")
cam_df = pdftotext_parser.get_rows(NEW_PDF)

In [4]:
ptt_df.equals(cam_df)

True

## Распознание таблиц в pdf

In [5]:
# вспомогательная функция для
# визуального контроля за результатами
# заспознания таблиц
mrgn = 50
font_scale = .35
thickness = 1
font = cv2.FONT_HERSHEY_SIMPLEX
line_type = cv2.LINE_AA

def make_pdf_image(file_path,
                   parse_method):

    parser = PDFparser(parse_method=parse_method)
    df_marked = parser.get_rows_marked(file_path)

    longest_str = df_marked.loc[df_marked[0].str.len().idxmax()][0]

    (w, h), baseline = cv2.getTextSize(longest_str,
                                       font,
                                       font_scale,
                                       thickness)

    line_height = h + baseline

    image = np.zeros((len(df_marked) * line_height + mrgn * 2,
                      w + mrgn * 2,
                      3), dtype="uint8")

    x, y0 = (mrgn, mrgn)
    for r in range(len(df_marked)):
        line = df_marked.loc[r, 0]
        y = y0 + r * line_height
        if df_marked.loc[r, "mark"] == "tbl_row":
            cv2.putText(image,
                        line,
                        (x, y),
                        font,
                        font_scale,
                        (255, 0, 0),
                        thickness,
                        line_type)
        elif df_marked.loc[r, "mark"] == "tbl_hdr":
            cv2.putText(image,
                        line,
                        (x, y),
                        font,
                        font_scale,
                        (255, 255, 0),
                        thickness,
                        line_type)
        else:
            cv2.putText(image,
                        line,
                        (x, y),
                        font,
                        font_scale,
                        (255, 255, 255),
                        thickness,
                        line_type)
    return image

In [6]:
# образцы файлов
pdf_pths = [str(pth) for pth
            in Path("../../task_description/examples/").iterdir()
            if pth.suffix == ".pdf"]

### Сравнение итогов работы PDFparser
на всех образцах после парсинга каждым из доступных методов

In [ ]:
parse_mtds = ["pdftotext", "camelot"]

params = {"axes.titlesize": "X-large"}
plt.rcParams.update(params)

fig, ax = plt.subplots(len(pdf_pths), len(parse_mtds),
                       figsize=(30, 70))

for i, file in enumerate(pdf_pths):
    for j, mtd in enumerate(parse_mtds):
        ax[i, j].axis("off")
        ax[i, j].title.set_text(os.path.basename(pdf_pths[i]) +
                                " (" + parse_mtds[j] + ")"  )
        ax[i, j].imshow(make_pdf_image(pdf_pths[i], parse_mtds[j]))
        fig.tight_layout()

